In [1]:
#Author: Miguel S. Chique Sayre / Resource and Modeling Geologist

import pyodbc
import pandas as pd
import os

import warnings
warnings.simplefilter("ignore", UserWarning)

In [2]:
#Código auxiliar para obtener la lista de logueos en excel

ruta = "C:/Users/PROPIETARIO/Py_MigueSCS/Py_Join_SQLDB_GVMapper/Daily_Report/Logueo_Excel"
Lit_InExe=pd.concat([pd.read_excel(os.path.join(ruta, fname), sheet_name='GvM_Lit')
                    for fname in os.listdir(ruta)], ignore_index=True)

In [3]:
#SQL SERVER Database
conn_sql = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-VMEQK3V\SQLEXPRESS;'
                      'Database=GVDATA1;'
                      'Trusted_Connection=yes;')

GvM_Collar = pd.read_sql_query('SELECT * FROM DATSSONDAJES', conn_sql)
GvM_Survey = pd.read_sql_query('SELECT * FROM DATSSURVEY', conn_sql)

GvM_Lit = pd.read_sql_query('SELECT * FROM GvM_Lit', conn_sql)
GvM_Lit["Lenght_m"]= round(GvM_Lit["Lenght_m"],2)

GvM_Alt = pd.read_sql_query('SELECT * FROM GvM_Alt', conn_sql)
GvM_Alt["Lenght_m"]= round(GvM_Alt["Lenght_m"],2)

GvM_Min = pd.read_sql_query('SELECT * FROM GvM_Min', conn_sql)
GvM_Min["Lenght_m"]= round(GvM_Min["Lenght_m"],2)
GvM_Min=GvM_Min.sort_values(['Hole_ID','Depth_From'])

GvM_Est = pd.read_sql_query('SELECT * FROM GvM_Est', conn_sql)
GvM_Est["Lenght_m"]= round(GvM_Est["Lenght_m"],2)

In [4]:
#Configuración de campos del Collar y Survey para exportación

GvM_Collar_Filter=GvM_Collar[["IDSONDAJE","ESTE","NORTE","COTACOLLAR","LARGO","OBSERVACIONES","FECHA_INICIO",
                             "FECHA_TERMINO","SECTOR"]]
GvM_Collar_Filter=GvM_Collar_Filter.rename(columns={'IDSONDAJE':'Hole_ID','ESTE':'East','NORTE':'North','COTACOLLAR':'Elevation'
                                                    ,'LARGO':'Depth_Total','OBSERVACIONES':'Estate',
                                                    'FECHA_INICIO':'Date_Start','FECHA_TERMINO':'Date_End','SECTOR':'Zone'})

GvM_Survey_Filter=GvM_Survey[["IDSONDAJE","PROFUNDIDAD","AZIMUTH","INCLINACION"]]
GvM_Survey_Filter=GvM_Survey_Filter.rename(columns={'IDSONDAJE':'Hole_ID','PROFUNDIDAD':'Depth','AZIMUTH':'Azimuth'
                                                    ,'INCLINACION':'Dip'})

Collar_GvM_Aux = GvM_Collar_Filter[["Hole_ID","Zone"]]
GvM_Survey_Filter = pd.merge(GvM_Survey_Filter, Collar_GvM_Aux, on='Hole_ID')

In [5]:
#Lista de sondajes a exportar

sondajes_excel = list(Lit_InExe['Hole_ID'].unique())

sondajes_tablet = ['23DDM2-9','23TA-DD98','23DDM3-4','23DDM3-6'] #Esta lista llenarla manualmente, son los sondajes no logueados con excel

sondajes_export = sondajes_excel + sondajes_tablet
sondajes_export

['23DD14-10',
 '23DD14-11',
 '23DD14-3',
 '23DD14-8',
 '23DD34E-01',
 '23DDM2-11',
 '23DDM2-14',
 '23DDM2-15',
 '23DDM2-2',
 '23DDM3-5',
 '23TA-DD109',
 '23DDM2-9',
 '23TA-DD98',
 '23DDM3-4',
 '23DDM3-6']

In [6]:
#Filtrado de los sondajes de reporte diario

Lit = GvM_Lit[GvM_Lit.Hole_ID.isin(sondajes_export)]
Alt = GvM_Alt[GvM_Alt.Hole_ID.isin(sondajes_export)]
Min = GvM_Min[GvM_Min.Hole_ID.isin(sondajes_export)]
Est = GvM_Est[GvM_Est.Hole_ID.isin(sondajes_export)]

Lit = Lit.sort_values(['Hole_ID', 'Depth_From'])
Alt = Alt.sort_values(['Hole_ID', 'Depth_From'])
Min = Min.sort_values(['Hole_ID', 'Depth_From'])
Est = Est.sort_values(['Hole_ID', 'Depth_From'])

In [7]:
#Verificación opcional de los sondajes

print(Lit['Hole_ID'].unique())
print(Alt['Hole_ID'].unique())
print(Min['Hole_ID'].unique())
print(Est['Hole_ID'].unique())

['23DD14-10' '23DD14-11' '23DD14-3' '23DD14-8' '23DD34E-01' '23DDM2-11'
 '23DDM2-14' '23DDM2-15' '23DDM2-2' '23DDM2-9' '23DDM3-4' '23DDM3-5'
 '23DDM3-6' '23TA-DD109' '23TA-DD98']
['23DD14-10' '23DD14-11' '23DD14-3' '23DD14-8' '23DD34E-01' '23DDM2-11'
 '23DDM2-14' '23DDM2-15' '23DDM2-2' '23DDM2-9' '23DDM3-4' '23DDM3-5'
 '23DDM3-6' '23TA-DD109' '23TA-DD98']
['23DD14-10' '23DD14-11' '23DD14-3' '23DD14-8' '23DD34E-01' '23DDM2-11'
 '23DDM2-14' '23DDM2-15' '23DDM2-2' '23DDM2-9' '23DDM3-4' '23DDM3-5'
 '23DDM3-6' '23TA-DD109' '23TA-DD98']
['23DD14-10' '23DD14-11' '23DD14-3' '23DD34E-01' '23DDM2-11' '23DDM2-14'
 '23DDM2-15' '23DDM2-2' '23DDM2-9' '23DDM3-4' '23DDM3-5' '23DDM3-6'
 '23TA-DD109' '23TA-DD98']


In [8]:
#Guardado del compilado

writer = pd.ExcelWriter('Export_Advance.xlsx')

Lit.to_excel(writer, sheet_name="Lit", index=False)
Alt.to_excel(writer, sheet_name="Alt", index=False)
Min.to_excel(writer, sheet_name="Min", index=False)
Est.to_excel(writer, sheet_name="Est", index=False)

writer.save()
writer.close()